## Data

In [ ]:
"""Example dataset with one sample for two datasets"""

from cot import Collection
coll = Collection.load_thoughtsource_100(names=['worldtree'],load_pregenerated_cots=True) #random_sample=False?
coll = coll.select(split="all", number_samples=1)
coll.select_generated_cots(cot_trigger = "kojima-01", api_service='cohere') #have one


In [20]:
from cot import Collection

coll = Collection("worldtree", load_pregenerated_cots=True,generate_mode='recache')
coll = coll.select(split="test", number_samples=1)
coll.select_generated_cots(cot_trigger = "kojima-01", api_service='cohere') 

Loading worldtree...


Generating train split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

Generating validation split: 0 examples [00:00, ? examples/s]

Dataset worldtree_dataset downloaded and prepared to /Users/robertpraas/.cache/huggingface/datasets/worldtree_dataset/thoughtsource/1.0.0/4ec0cd827b41f05891af9a27bf461fecd407e2fe7c1beebfed1eb00193c2cd52. Subsequent calls will reuse this data.


  0%|          | 0/3 [00:00<?, ?it/s]

In [21]:
coll

| Name      | Train   | Valid   |   Test |
|-----------|---------|---------|--------|
| worldtree | -       | -       |      1 |

Not loaded: ['aqua', 'asdiv', 'commonsense_qa', 'entailment_bank', 'gsm8k', 'mawps', 'med_qa', 'med_qa_open', 'medmc_qa', '_init_', 'mmlu_clinical_knowledge', 'mmlu_college_biology', 'mmlu_college_medicine', 'mmlu_medical_genetics', 'mmlu_professional_medicine', '_init_', 'mmlu_anatomy', 'open_book_qa', 'pubmed_qa', 'qed', 'strategy_qa', 'svamp']

In [2]:
coll['worldtree']['test'][0]

{'id': 'worldtree_test_788',
 'ref_id': '',
 'question': 'When light hits a mirror, most of the light is',
 'type': 'multiplechoice',
 'choices': ['refracted.', 'reflected.', 'absorbed.', 'transmitted.'],
 'context': '',
 'cot': ['A mirror reflects light.',
  'Reflection is when a wave bounces off a surface and travels in the opposite direction relative to the angle of incidence.',
  'Light is a kind of wave.',
  'When light hits a reflective object , that light bounces off that object.',
  'A mirror is a kind of reflective object.'],
 'answer': ['reflected.'],
 'generated_cot': [],
 'feedback': []}

## Generate and extract

In [3]:
from langchain.llms import OpenAI 
from langchain.prompts import PromptTemplate 
from langchain.chains.llm import LLMChain
import json
from langchain.chat_models import ChatOpenAI


In [4]:
"""CoT Chain"""

llm = ChatOpenAI(temperature=.0,model_name="gpt-3.5-turbo") #ADA #for chat: gpt-3.5-turbo




template = """{instruction}

Question: {question}
Answer_choices: {answer_choices}

{cot_trigger}
"""

prompt_template = PromptTemplate(input_variables=["instruction","question","answer_choices","cot_trigger"], template=template)
cot_chain = LLMChain(llm=llm, prompt=prompt_template,output_key="cot")

In [5]:
"""answer extraction chain"""

extraction_template = """{instruction}

Question: {question}
Answer_choices: {answer_choices}

Cot: {cot_trigger}{cot}
{answer_extraction}
"""

prompt_template = PromptTemplate(input_variables=["instruction","question","answer_choices","cot_trigger","cot","answer_extraction"], template=extraction_template)
answer_chain = LLMChain(llm=llm, prompt=prompt_template,output_key="predicted_answer")


In [6]:
"""CoT-Ans_extraction chain"""

from langchain.chains import SequentialChain
overall_chain = SequentialChain(chains=[cot_chain, answer_chain],
                                input_variables=["instruction","question","answer_choices","cot_trigger","answer_extraction"],
                                output_variables=["cot", "predicted_answer"],
                                verbose=True)

In [22]:
"""Generate CoT with use of TS-schema"""
#compare with config used before; what about max tokens?
#config contains what the chain needs
input_dict = {
    "instruction": "Be faithful and a little hopeful",
    "cot_trigger": "Answer: Let's think step by step.",
    "answer_extraction": "Therefore, among A through D, the answer is",
    'model':"gpt-3.5-turbo",
    'temperature': 0,
    'max_tokens': 800 
}
coll = coll.generate_extract_flexible(chain=overall_chain,input_dict=input_dict,name='worldtree',split='test')




Generating worldtree...


> Entering new SequentialChain chain...

> Finished chain.


In [23]:
coll

[{'id': 'worldtree_test_788',
  'ref_id': '',
  'question': 'When light hits a mirror, most of the light is',
  'type': 'multiplechoice',
  'choices': ['refracted.', 'reflected.', 'absorbed.', 'transmitted.'],
  'context': '',
  'cot': ['A mirror reflects light.',
   'Reflection is when a wave bounces off a surface and travels in the opposite direction relative to the angle of incidence.',
   'Light is a kind of wave.',
   'When light hits a reflective object , that light bounces off that object.',
   'A mirror is a kind of reflective object.'],
  'answer': ['reflected.'],
  'generated_cot': [{'id': 'be724991-87a8-400d-b8b8-b4664bc1f400',
    'fragments_version': '0.01',
    'instruction': 'Be faithful and a little hopeful',
    'cot_trigger': "Answer: Let's think step by step.",
    'cot_trigger_template': '',
    'prompt_text': '',
    'cot': 'When light hits a mirror, it bounces back off the surface of the mirror. This process is called reflection. Therefore, the correct answer is B

In [11]:
second_coll = Collection.to_Collection(coll,"worldtree",'test','file_test')


In [24]:
worldtree_2 = {'worldtree':{'test':coll}}

In [25]:
#after recache
with open("sample.json", "w") as outfile:
    json.dump(worldtree_2, outfile)
collect = Collection.from_json('sample.json')



[[{'id': 'be724991-87a8-400d-b8b8-b4664bc1f400',
   'fragments_version': '0.01',
   'instruction': 'Be faithful and a little hopeful',
   'cot_trigger': "Answer: Let's think step by step.",
   'cot_trigger_template': '',
   'prompt_text': '',
   'cot': 'When light hits a mirror, it bounces back off the surface of the mirror. This process is called reflection. Therefore, the correct answer is B) reflected.',
   'answers': [{'id': '588696ec-71b0-41c4-a894-89a9cc0947c4',
     'answer_extraction': 'Therefore, among A through D, the answer is',
     'answer_extraction_template': '',
     'answer_extraction_text': '',
     'answer': 'B) reflected.',
     'answer_from_choices': '',
     'correct_answer': None}],
   'author': '',
   'date': '2023/05/07 13:31:44',
   'api_service': '',
   'model': "{'name': 'gpt-3.5-turbo', 'temperature': 0, 'max_tokens': 800}",
   'comment': 'generated and extracted',
   'annotations': []}]]

In [27]:
collect.evaluate()

  0%|          | 0/1 [00:00<?, ?ex/s]

{'worldtree': {'test': {'accuracy': {'gpt-3.5-turbo': {"Be faithful and a little hopeful_Answer: Let's think step by step._Therefore, among A through D, the answer is": 1.0}}}}}

In [19]:
#before recache
with open("sample.json", "w") as outfile:
    json.dump(worldtree_2, outfile)
collect = Collection.from_json('sample.json')

collect['worldtree']['test']['generated_cot']

KeyError: 'answer_from_choices'

In [29]:
#extra try
coll = Collection("worldtree", load_pregenerated_cots=True)
coll = coll.select(split="test", number_samples=1)
coll.select_generated_cots(cot_trigger = "kojima-01", api_service='cohere') 
coll.generate_extract_flexible(chain=overall_chain,input_dict=input_dict,name='worldtree')


Reusing dataset worldtree_dataset (/Users/robertpraas/.cache/huggingface/datasets/worldtree_dataset/thoughtsource/1.0.0/4ec0cd827b41f05891af9a27bf461fecd407e2fe7c1beebfed1eb00193c2cd52)


Loading worldtree...


  0%|          | 0/3 [00:00<?, ?it/s]

Generating worldtree...


> Entering new SequentialChain chain...

> Finished chain.


In [30]:
coll

AttributeError: 'list' object has no attribute 'num_rows'

In [18]:
collect['worldtree']

DatasetDict({
    test: Dataset({
        features: ['id', 'ref_id', 'question', 'type', 'choices', 'context', 'cot', 'answer', 'generated_cot', 'feedback'],
        num_rows: 1
    })
})

In [21]:
third_coll = Collection("worldtree", load_pregenerated_cots=True)
third_coll

Reusing dataset worldtree_dataset (/Users/robertpraas/.cache/huggingface/datasets/worldtree_dataset/thoughtsource/1.0.0/4ec0cd827b41f05891af9a27bf461fecd407e2fe7c1beebfed1eb00193c2cd52)


Loading worldtree...


  0%|          | 0/3 [00:00<?, ?it/s]

ValueError: 4 columns passed, passed data had 3 columns

In [19]:
third_coll = Collection("worldtree", load_pregenerated_cots=True)
third_coll = third_coll.select(split="test", number_samples=1)
third_coll.select_generated_cots(cot_trigger = "kojima-01", api_service='cohere') 

Reusing dataset worldtree_dataset (/Users/robertpraas/.cache/huggingface/datasets/worldtree_dataset/thoughtsource/1.0.0/4ec0cd827b41f05891af9a27bf461fecd407e2fe7c1beebfed1eb00193c2cd52)


Loading worldtree...


  0%|          | 0/3 [00:00<?, ?it/s]

In [20]:
third_coll

AttributeError: 'str' object has no attribute 'self'

In [13]:
type(coll)
coll
#need to have dataset and potentially split

AttributeError: 'list' object has no attribute 'num_rows'

In [12]:
worldtree_10 = Collection.from_json("worldtree_10.json")
type(worldtree_10)

cot.dataloader.Collection

In [16]:
type(coll)

cot.dataloader.Collection

In [9]:
json.dump(coll, 'test.json') 

TypeError: Object of type Collection is not JSON serializable

In [14]:
#Force langchain into TS structure 
ts_set = {'worldtree':{'test':coll}} #gives error


In [17]:
type(coll)

cot.dataloader.Collection

In [20]:
coll

AttributeError: 'list' object has no attribute 'num_rows'

In [ ]:

#create and collect a json to make collection
with open(f"{file_name}.json", "w") as outfile:
    json.dump(ts_set, outfile) 
    #ts_set.dump(outfile) #dump dict not possible
collect = Collection.from_json(f'{file_name}.json')

AttributeError: 'dict' object has no attribute 'dump'

In [29]:
coll['worldtree']['test'][0]['generated_cot'][27]

{'id': '2763c9a3-0b71-45ce-b5e3-33c9334559c0',
 'fragments_version': '0.01',
 'instruction': 'Be faithful and a little hopeful',
 'cot_trigger': "Answer: Let's think step by step.",
 'cot_trigger_template': '',
 'prompt_text': '',
 'cot': 'A) Is the mass of Earth a factor in causing gravity on Earth? Yes, it is. The more massive an object, the stronger its gravitational pull. So, option A is a possible answer.\n\nB) Is the rotation of Earth a factor in causing gravity on Earth? No, it is not. The rotation of Earth affects other phenomena, such as day and night, but not gravity. So, option B is not a correct answer.\n\nC) Is the revolution of Earth a factor in causing gravity on Earth? No, it is not. The revolution of Earth around the Sun affects the seasons, but not gravity. So, option C is not a correct answer.\n\nD) Are weather patterns on Earth a factor in causing gravity on Earth? No, they are not. Weather patterns are caused by atmospheric conditions, but they do not affect gravit

## Generate or extract

In [1]:
from cot import Collection
new_coll = Collection("worldtree", load_pregenerated_cots=True)
new_coll = new_coll.select(split="test", number_samples=1)

new_coll.select_generated_cots(cot_trigger = "kojima-01", api_service='cohere') 


SyntaxError: invalid syntax (dataloader.py, line 151)

In [43]:
new_coll

| Name      | Train   | Valid   |   Test |
|-----------|---------|---------|--------|
| worldtree | -       | -       |      1 |

Not loaded: ['aqua', 'asdiv', 'commonsense_qa', 'entailment_bank', 'gsm8k', 'mawps', 'med_qa', 'medmc_qa', 'open_book_qa', 'pubmed_qa', 'qed', 'strategy_qa', 'svamp']

In [46]:
new_coll.generate_flexible(chain=cot_chain,input_dict=input_dict,name='worldtree',split='test')
new_coll

Generating worldtree...


In [2]:
#from langchain.llms.openai import OpenAIChat

from langchain.llms import OpenAI 
from langchain.prompts import PromptTemplate 
from langchain.chains.llm import LLMChain
from cot import Collection
import json

In [23]:
"""Extract script: Assumes there are CoTs in the dataset already"""

input_dict = {
    'instruction': None,
    "answer_extraction": "Be faithful and a little hopeful",
    "cot_trigger": "Answer: Let's think step by step.",
    'api_service': "chat_openai",
    'model':"gpt-3.5-turbo",
    'temperature': 0,
    'max_tokens': 800 
}
new_coll.extract_flexible(chain=answer_chain,input_dict=input_dict)

Generating worldtree...


IndexError: list index out of range

In [ ]:
coll['strategy_qa']['train'][0]['generated_cot']

In [38]:
coll['worldtree']['test'][0]['generated_cot'][1]

{'id': '6b3d0b3b-bc87-4ecb-a271-ee7d181cc2a2',
 'fragments_version': '0.01',
 'instruction': None,
 'cot_trigger': "Answer: Let's think step by step.",
 'cot_trigger_template': '',
 'prompt_text': '',
 'cot': "\n1. The mass of Earth causes a gravitational attraction between Earth and you.\n2. The rotation of Earth causes centrifugal force, which is a pseudo-force that appears to be a reaction force in a rotating reference frame.\n3. The revolution of Earth around the Sun causes Earth to be pulled by the Sun's gravity.\n4. Weather patterns on Earth do not cause gravity.\nThe correct answer is A) the mass of Earth.",
 'answers': [{'id': '65a672ce-5b05-4c74-aaac-5a05051c66a2',
   'answer_extraction': 'Be faithful and a little hopeful',
   'answer_extraction_template': '',
   'answer_extraction_text': '',
   'answer': 'as you approach each question, and use logical reasoning to eliminate incorrect answer choices.',
   'correct_answer': None}],
 'author': '',
 'date': '2023/04/12 11:06:02',

In [24]:
"""Generate only"""
input_dict = {
    "instruction": "Be faithful and a little hopeful",
    "cot_trigger": "Answer: Let's think step by step.",
    'api_service': "chat_openai",
    'model': "gpt-3.5-turbo",
    "answer_extraction": "Therefore, among A through D, the answer is",
    'temperature': 0,
    'max_tokens': 800  
}

new_coll.generate_flexible(chain=cot_chain,input_dict=input_dict)

Generating worldtree...
Generating worldtree...
Generating worldtree...


In [ ]:
coll['worldtree']['test'][0]['generated_cot'][2]

## Reflection

In [42]:
llm = ChatOpenAI(temperature=.0,model_name="gpt-3.5-turbo")

reflect_template = """
    Question: {question}
    Answer_choices: {answer_choices}

    Cot: {cot_trigger}{cot}
    {answer_extraction}
    Answer: {answer}
    
    {reflection_prompt}
    """
prompt_template = PromptTemplate(input_variables=["question","answer_choices","cot_trigger","cot","answer_extraction",'answer','reflection_prompt'], template=reflect_template)
reflect_chain = LLMChain(llm=llm, prompt=prompt_template,output_key="reflection")

extraction_template = """{instruction}

    Question: {question}
    Answer_choices: {answer_choices}

    Cot: {cot_trigger}{cot}
    {answer_extraction}{answer}
    {reflection_prompt}{reflection}

    {reflect_answer_extraction}
    """
    #Get reflection
prompt_template = PromptTemplate(input_variables=["instruction","question","answer_choices","cot_trigger","cot","answer_extraction",'answer','reflection_prompt','reflection','reflect_answer_extraction'], template=extraction_template)
reflect_answer_chain = LLMChain(llm=llm, prompt=prompt_template,output_key="reflection_answer")

    # This is the overall chain where we run these two chains in sequence.
from langchain.chains import SequentialChain
reflect_overall_chain = SequentialChain(chains=[reflect_chain, reflect_answer_chain],input_variables=["instruction","question","answer_choices","cot_trigger","answer_extraction",'cot','answer','reflection_prompt','reflect_answer_extraction'],
        output_variables=["reflection", "reflection_answer"],
        verbose=True)




In [45]:
#check for what is already in item
input_dict = {
    'cot_trigger':"", 
    'answer':"", 
    'answer_extraction': "", 
    'cot': "", 
    'instruction': "",
    'api_service': "chat_openai", 
    'model': "gpt-3.5-turbo",
    'reflection_prompt':"Double check this",
    'reflect_answer_extraction':'Based on the reflection, what is the definite answer?',
    'temperature': 0,
    'max_tokens': 800 

}
coll.metareason_flexible(chain=reflect_overall_chain,input_dict=input_dict)

Generating worldtree...


> Entering new SequentialChain chain...

> Finished chain.


In [46]:
coll['worldtree']['test'][0]['generated_cot'][1]

{'id': 'ddbcca69-1ded-4593-a80f-03f3644190db',
 'fragments_version': '0.01',
 'instruction': '',
 'cot_trigger': 'Double check this',
 'cot_trigger_template': '',
 'prompt_text': '',
 'cot': 'Confirmed, the correct answer is A) the mass of Earth.',
 'answers': [{'id': 'ea0f47f5-c8f8-41ee-ada0-f59618e8748f',
   'answer_extraction': 'Based on the reflection, what is the definite answer?',
   'answer_extraction_template': '',
   'answer_extraction_text': 'self_reflection',
   'answer': 'The definite answer is A) the mass of Earth.',
   'correct_answer': None}],
 'author': '',
 'date': '2023/04/12 11:09:03',
 'api_service': 'chat_openai',
 'model': "{'name': 'gpt-3.5-turbo', 'temperature': 0, 'max_tokens': 800}",
 'comment': 'self_reflection cot',
 'annotations': []}

## Experiments

In [7]:
from cot import Collection
col = Collection.load_thoughtsource_100(load_pregenerated_cots=True)
col.select_generated_cots(author='thoughtsource', cot_trigger='kojima-01')
dataset = col['worldtree']


[nltk_data] Downloading package punkt to
[nltk_data]     /Users/robertpraas/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


  0%|          | 0/100 [00:00<?, ?ex/s]

  0%|          | 0/100 [00:00<?, ?ex/s]

  0%|          | 0/100 [00:00<?, ?ex/s]

  0%|          | 0/100 [00:00<?, ?ex/s]

  0%|          | 0/100 [00:00<?, ?ex/s]

  0%|          | 0/100 [00:00<?, ?ex/s]

In [8]:
worldtree = Collection(["worldtree"], verbose=False)
worldtree_1 = worldtree.select(split="train", number_samples=1, random_samples=True, seed=0)

Loading worldtree...


In [55]:
worldtree_1['worldtree']['train'][0]

{'id': '1577',
 'ref_id': '',
 'question': 'A parent and a child share several characteristics. Both individuals are tall, have curly hair, are good cooks, and have freckles. Which of these characteristics is a learned behavior?',
 'type': 'multiplechoice',
 'choices': ['being tall',
  'having curly hair',
  'being a good cook',
  'having freckles'],
 'context': '',
 'cot': ['Skills are learned characteristics.',
  'A behavior is a kind of characteristic.',
  'Cooking is a kind of skill for preparing food.'],
 'answer': ['being a good cook'],
 'generated_cot': [],
 'feedback': []}

Reflect

Generate and extract

In [9]:
input_dict = {
    "instruction": "Be faithful and a little hopeful",
    "cot_trigger": "Answer: Let's think step by step.",
    "answer_extraction": "Therefore, among A through D, the answer is" 
}

In [10]:
generate_test = worldtree_1.generate_extract_flexibly(chain=overall_chain,input_dict=input_dict)

Generating worldtree...


> Entering new SequentialChain chain...

> Finished chain.
processed_example:
{'id': '1577', 'ref_id': '', 'question': 'A parent and a child share several characteristics. Both individuals are tall, have curly hair, are good cooks, and have freckles. Which of these characteristics is a learned behavior?', 'type': 'multiplechoice', 'choices': ['being tall', 'having curly hair', 'being a good cook', 'having freckles'], 'context': '', 'cot': ['Skills are learned characteristics.', 'A behavior is a kind of characteristic.', 'Cooking is a kind of skill for preparing food.'], 'answer': ['being a good cook'], 'generated_cot': [{'id': '2db4a9e4-301c-4197-8796-b40931fd7d6f', 'fragments_version': '0.01', 'instruction': 'Be faithful and a little hopeful', 'cot_trigger': "Answer: Let's think step by step.", 'cot_trigger_template': '', 'prompt_text': '', 'cot': '\n1. A) is a learned behavior.\n\n2. B) is a learned behavior.\n\n3. C) is a learned behavior.\n\n4. D) is a lea

In [14]:
#Force langchain into TS structure 
worldtree_new = {'worldtree':{'train':generate_test}}

#create and collect a json to make collection
with open("sample.json", "w") as outfile:
    json.dump(worldtree_new, outfile)
collect = Collection.from_json('sample.json')

collect['worldtree']['train']['generated_cot']

[[{'id': '2db4a9e4-301c-4197-8796-b40931fd7d6f',
   'fragments_version': '0.01',
   'instruction': 'Be faithful and a little hopeful',
   'cot_trigger': "Answer: Let's think step by step.",
   'cot_trigger_template': '',
   'prompt_text': '',
   'cot': '\n1. A) is a learned behavior.\n\n2. B) is a learned behavior.\n\n3. C) is a learned behavior.\n\n4. D) is a learned behavior.\n\n5. E) is a learned behavior.\n\n6. F) is a learned behavior.\n\n7. G) is a learned behavior.\n\n8. H) is a learned behavior.\n\n9. I) is a learned behavior.\n\n10. J) is a learned behavior.\n\n11. K) is a learned behavior.\n\n12. L) is a learned behavior.\n\n13. M) is a learned behavior.\n\n14. N) is a learned behavior.\n\n15. O) is a learned behavior.\n\n16. P) is a learned behavior.\n\n17. Q) is a learned behavior.\n\n18. R) is a learned behavior.\n\n19. S) is a learned behavior.\n\n20. T) is a learned behavior.\n\n21. U) is a learned behavior.\n\n22. V) is a learned behavior.\n\n23. W) is a learned behavio

Generate

In [17]:
generate_only = worldtree_1.generate_flexible(chain=cot_chain,input_dict=input_dict)
generate_only

Generating worldtree...
processed_example:
{'id': '1577', 'ref_id': '', 'question': 'A parent and a child share several characteristics. Both individuals are tall, have curly hair, are good cooks, and have freckles. Which of these characteristics is a learned behavior?', 'type': 'multiplechoice', 'choices': ['being tall', 'having curly hair', 'being a good cook', 'having freckles'], 'context': '', 'cot': ['Skills are learned characteristics.', 'A behavior is a kind of characteristic.', 'Cooking is a kind of skill for preparing food.'], 'answer': ['being a good cook'], 'generated_cot': [{'id': '7d977578-5359-49d9-bd26-c373911bf3c9', 'fragments_version': '0.01', 'instruction': 'Be faithful and a little hopeful', 'cot_trigger': "Answer: Let's think step by step.", 'cot_trigger_template': '', 'prompt_text': '', 'cot': '\n1. A) is a learned behavior.\n\n2. B) is a learned behavior.\n\n3. C) is a learned behavior.\n\n4. D) is a learned behavior.\n\n5. E) is a learned behavior.\n\n6. F) is a 

[{'id': '1577',
  'ref_id': '',
  'question': 'A parent and a child share several characteristics. Both individuals are tall, have curly hair, are good cooks, and have freckles. Which of these characteristics is a learned behavior?',
  'type': 'multiplechoice',
  'choices': ['being tall',
   'having curly hair',
   'being a good cook',
   'having freckles'],
  'context': '',
  'cot': ['Skills are learned characteristics.',
   'A behavior is a kind of characteristic.',
   'Cooking is a kind of skill for preparing food.'],
  'answer': ['being a good cook'],
  'generated_cot': [{'id': '7d977578-5359-49d9-bd26-c373911bf3c9',
    'fragments_version': '0.01',
    'instruction': 'Be faithful and a little hopeful',
    'cot_trigger': "Answer: Let's think step by step.",
    'cot_trigger_template': '',
    'prompt_text': '',
    'cot': '\n1. A) is a learned behavior.\n\n2. B) is a learned behavior.\n\n3. C) is a learned behavior.\n\n4. D) is a learned behavior.\n\n5. E) is a learned behavior.\n

Extract

In [18]:
#Force langchain into TS structure -- from geneate
worldtree_new = {'worldtree':{'train':generate_only}}

#create and collect a json to make collection
with open("sample.json", "w") as outfile:
    json.dump(worldtree_new, outfile)
collect = Collection.from_json('sample.json')

In [23]:
extract = collect.extract_flexible(chain=answer_chain,input_dict=input_dict)

Generating worldtree...
processed_example:
{'id': '1577', 'ref_id': '', 'question': 'A parent and a child share several characteristics. Both individuals are tall, have curly hair, are good cooks, and have freckles. Which of these characteristics is a learned behavior?', 'type': 'multiplechoice', 'choices': ['being tall', 'having curly hair', 'being a good cook', 'having freckles'], 'context': '', 'cot': ['Skills are learned characteristics.', 'A behavior is a kind of characteristic.', 'Cooking is a kind of skill for preparing food.'], 'answer': ['being a good cook'], 'generated_cot': [{'id': '7d977578-5359-49d9-bd26-c373911bf3c9', 'fragments_version': '0.01', 'instruction': 'Be faithful and a little hopeful', 'cot_trigger': "Answer: Let's think step by step.", 'cot_trigger_template': '', 'prompt_text': '', 'cot': '\n1. A) is a learned behavior.\n\n2. B) is a learned behavior.\n\n3. C) is a learned behavior.\n\n4. D) is a learned behavior.\n\n5. E) is a learned behavior.\n\n6. F) is a 

In [33]:
coll = Collection.load_thoughtsource_100(names='worldtree',load_pregenerated_cots=True)
coll = coll.select(split="all", number_samples=1)
coll.select_generated_cots(author='thoughtsource',cot_trigger = None)